In [55]:
import numpy as np
import pandas as pd
import torch
from torch import nn
import os

In [56]:
df=pd.read_csv(r'C:\Users\Avinash mishra\Desktop\email-spam-detection\src\model\data\combined_enron.csv')

In [57]:
df

,label,label_num,text
0,ham,0,subject fw weather sites this is the list of w...
1,spam,1,subject i cant believe the changma wow wow wow...
2,spam,1,subject 4 color printing special request addit...
3,spam,1,subject nobody can beat us poloponybreakaway i...
4,ham,0,subject energy extravaganza 2 weeks away energ...
...,...,...,...
33697,ham,0,subject enron mentions enron jolt investments ...
33698,ham,0,subject hpl meter 981295 hungerford grain comp...
33699,ham,0,subject re expenses tammie has the receipts fr...
33700,spam,1,subject comparison report ciearance bundle 1 w...


In [58]:
df.shape

(33702, 3)

In [59]:
count_spam=(df["label"]=="spam").sum()
print((count_spam/df.shape[0])*100)

50.90795798468933


In [60]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
df = df.drop_duplicates(subset="text").reset_index(drop=True)
X=df["text"]
y=df["label_num"]
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

vectorizer = TfidfVectorizer(max_features=5000)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

In [61]:
X_train_vec.shape

(24122, 5000)

In [62]:
duplicates = set(X_train) & set(X_test)
print(len(duplicates))   # should be 0


0


In [63]:
device= torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [64]:
class NNModel(nn.Module):
    def __init__(self, input_features, hidden_unit=8):
        super().__init__()
        self.layer1= nn.Linear(in_features=input_features, out_features=hidden_unit)
        self.layer2= nn.Linear(in_features=hidden_unit, out_features= hidden_unit)
        self.layer3= nn.Linear(in_features=hidden_unit, out_features=1)
        self.relu= nn.ReLU()
    def forward(self, x):
        return self.layer3(self.relu(self.layer2(self.relu(self.layer1(x)))))
model= NNModel(
    input_features= X_train_vec.shape[1],
    hidden_unit=64
).to(device)
model

NNModel(
  (layer1): Linear(in_features=5000, out_features=64, bias=True)
  (layer2): Linear(in_features=64, out_features=64, bias=True)
  (layer3): Linear(in_features=64, out_features=1, bias=True)
  (relu): ReLU()
)

In [65]:
loss_fn=nn.BCEWithLogitsLoss()
optimizer= torch.optim.SGD(params= model.parameters(), lr=0.01)

In [66]:
from torch.utils.data import TensorDataset, DataLoader

X_train_vec_t= torch.tensor(X_train_vec.toarray(), dtype=torch.float32)
X_test_vec_t= torch.tensor(X_test_vec.toarray(), dtype=torch.float32)
y_train_tensor= torch.tensor(y_train.values, dtype=torch.float32)
y_test_tensor= torch.tensor(y_test.values, dtype=torch.float32)

train_dataset = TensorDataset(X_train_vec_t, y_train_tensor)
test_dataset = TensorDataset(X_test_vec_t, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)


In [67]:
torch.manual_seed(42)
train_loss, train_acc=0,0
test_loss, test_acc=0,0

epochs=25
for epoch in range(epochs):
    model.train()
    train_loss=0
    train_acc=0
    for X,y in train_loader:
        X=X.to(device)
        y=y.to(device)
        pred_logits=model(X).squeeze()
        train_preds= torch.round(torch.sigmoid(pred_logits))
        loss=loss_fn(pred_logits, y)
        train_loss+=loss.item(
            
        )
        train_acc+= (train_preds==y).float().mean()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    
    model.eval()
    test_loss=0
    test_acc=0
    with torch.inference_mode():
        for X_test,y_test in test_loader:
            X_test=X_test.to(device)
            y_test=y_test.to(device)
            test_logits= model(X_test).squeeze()
            test_preds= torch.round(torch.sigmoid(test_logits))
            test_loss+=loss_fn(test_logits, y_test).item()
            
            test_acc+= (test_preds==y_test).float().mean()
        test_loss/=len(test_loader)
        test_acc/=len(test_loader)
    train_loss/=len(train_loader)
    train_acc/=len(train_loader)
    if epoch%2==0:
        print(f" epoch: {epoch} | train loss: {train_loss} | training_accuracy: {train_acc} | test loss: {test_loss} | test_acc: {test_acc}")

 epoch: 0 | train loss: 0.6923370873580245 | training_accuracy: 0.5179778337478638 | test loss: 0.6923991350269822 | test_acc: 0.5139660239219666
 epoch: 2 | train loss: 0.6910678610719484 | training_accuracy: 0.5180065631866455 | test loss: 0.6908260785713398 | test_acc: 0.5139660239219666
 epoch: 4 | train loss: 0.6875849780416615 | training_accuracy: 0.5180160999298096 | test loss: 0.6861649985035891 | test_acc: 0.5139660239219666
 epoch: 6 | train loss: 0.6727965961874954 | training_accuracy: 0.6005406975746155 | test loss: 0.6642036396990377 | test_acc: 0.7559303641319275
 epoch: 8 | train loss: 0.5695328177286396 | training_accuracy: 0.9588030576705933 | test loss: 0.5015136631077559 | test_acc: 0.9636023044586182
 epoch: 10 | train loss: 0.23117881418460243 | training_accuracy: 0.9677042961120605 | test loss: 0.18017445248428476 | test_acc: 0.9720568656921387
 epoch: 12 | train loss: 0.1080159992078413 | training_accuracy: 0.9766884446144104 | test loss: 0.10082348077385514 | te

In [68]:
model.eval()

all_preds = []
all_labels = []

with torch.inference_mode():
    for X, y in test_loader:
        logits = model(X).squeeze()
        preds = torch.round(torch.sigmoid(logits))
        all_preds.append(preds)
        all_labels.append(y)

all_preds = torch.cat(all_preds)
all_labels = torch.cat(all_labels)

print("Final Test Accuracy:",
      (all_preds == all_labels).float().mean())


Final Test Accuracy: tensor(0.9874)


In [69]:
print(X_train_vec_t.shape)
print(X_test_vec_t.shape)


torch.Size([24122, 5000])
torch.Size([6031, 5000])


In [70]:
df["label"].value_counts()

label
ham     15591
spam    14562
Name: count, dtype: int64

In [71]:
def pred(model: nn.Module,
        data: list):
    test_preds=[]
    model.eval()
    with torch.inference_mode():
        for sample in data:
            pred= model(sample)
            test_preds.append(pred)
    return torch.cat(test_preds)

In [72]:
torch.save(model.state_dict(), 'model.pth')

In [73]:
import joblib
joblib.dump(vectorizer, "tfidf.pkl")

['tfidf.pkl']